# SVD baseline algorithm using surprise package:

using `matrix_factorization.SVD` algorithm from http://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

In [3]:
import helpers
from surprise_helpers import CustomReader, get_ratings_from_predictions
from surprise import Reader, Dataset

## Data loading

In [4]:
reader = CustomReader()
filepath = helpers.get_train_file_path()
data = Dataset.load_from_file(filepath, reader=reader)

## Training and validation
